<a href="https://colab.research.google.com/github/RockstarAryan/Digit-recognizer-with-PyTorch/blob/main/Digit_rec_Aryan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Digit Recognizer with PyTorch**





In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torchvision.transforms import ToTensor
import torchvision.models as models

In [ ]:
#Data Loading: Load the MNIST dataset using PyTorch's data loading utilities.

training_data = datasets.MNIST(
    root="new",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="new",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 9912422/9912422 [00:00<00:00, 125497563.27it/s]

Extracting new/MNIST/raw/train-images-idx3-ubyte.gz to new/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 23012099.89it/s]


Extracting new/MNIST/raw/train-labels-idx1-ubyte.gz to new/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 61541875.98it/s]

Extracting new/MNIST/raw/t10k-images-idx3-ubyte.gz to new/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 18622217.76it/s]


Extracting new/MNIST/raw/t10k-labels-idx1-ubyte.gz to new/MNIST/raw



In [ ]:
#Using DataLoader to efficiently load and iterate over the dataset and define batches to 64 each

trainloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
#Creating an iterator and using it to also get the next batch of data

dataiterator = iter(trainloader)
images, labels = next(dataiterator)


In [ ]:
#checking the shape of the images and labels in one batch
print(images.shape)
print(labels.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [ ]:
#Creats a simple neural network using nn.Sequential
input_size = 784

# Define the sizes of hidden layers
hidden_sizes = [128, 128]

output_size = 10

model = nn.Sequential(
    #First linear layer
    nn.Linear(input_size, hidden_sizes[0]),
    nn.ReLU(),

    #Second linear layer
    nn.Linear(hidden_sizes[0], hidden_sizes[1]),
    nn.ReLU(),

    #Third linear layer
    nn.Linear(hidden_sizes[1], output_size),

    #usig logSoftMax fn
    nn.LogSoftmax(dim=1)
)

print(model)


Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [ ]:
criterion = nn.NLLLoss()
#takes ext batch of images and their labels
images, labels = next(iter(trainloader))
#flattens the input images
images = images.view(images.shape[0], -1)

logps = model(images) #forward pass
loss = criterion(logps, labels) #calculate the NLL loss


In [ ]:
#Initialises the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

#epoch is number of times we iterate over the training set
epochs = 25
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flattens images
        images = images.view(images.shape[0], -1)

        # Training pass
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        #backpropagating
        loss.backward()

        #optimizes weights here
        optimizer.step()

        running_loss += loss.item()
    else:
        print("Epoch ",e," - Training loss -",running_loss/len(trainloader))

Epoch  0  - Training loss - 0.9089385764494634
Epoch  1  - Training loss - 0.32324035198036544
Epoch  2  - Training loss - 0.261190792104837
Epoch  3  - Training loss - 0.21892754654131974
Epoch  4  - Training loss - 0.1874993307940932
Epoch  5  - Training loss - 0.1632655027356229
Epoch  6  - Training loss - 0.14392152082905776
Epoch  7  - Training loss - 0.12812125686206607
Epoch  8  - Training loss - 0.11459633768168784
Epoch  9  - Training loss - 0.1040819770724042
Epoch  10  - Training loss - 0.09403376452434165
Epoch  11  - Training loss - 0.08656485140947168
Epoch  12  - Training loss - 0.08008091929934617
Epoch  13  - Training loss - 0.07356259087448133
Epoch  14  - Training loss - 0.06826652032184179
Epoch  15  - Training loss - 0.06341000701358387
Epoch  16  - Training loss - 0.058978785699638506
Epoch  17  - Training loss - 0.05493889237120589
Epoch  18  - Training loss - 0.051569489346169776
Epoch  19  - Training loss - 0.04790104382427167
Epoch  20  - Training loss - 0.044

In [ ]:
correct_count, all_count = 0, 0

#loops over test dataset
for images,labels in testloader:
  for i in range(len(labels)):
    #flattens image
    img = images[i].view(1, 784)
    #Makes prediction
    with torch.no_grad():
        logps = model(img)

    #applies the exponential function to the log probabilities
    ps = torch.exp(logps)
    #converted to numpy array
    probab = list(ps.numpy()[0])
    #This finds the index of the maximum probability in the probab list
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    #For correct prediction
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("Model Accuracy =", 100 *  (correct_count/all_count),"%")

Number Of Images Tested = 10000
Model Accuracy = 97.86 %


In [ ]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 92.2MB/s]


In [ ]:
model = models.vgg16() # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
torch.save(model, 'model.pth')